In [ ]:
#NAME: M.SHRAVANI PRIYA
#USN: 21BTRCL059
#MINI PROJECT ON CINEMATCH

In [1]:
import wx  # Import wxPython library for GUI
from collections import defaultdict  # Import defaultdict for genre indexing
from heapq import nlargest  # Import nlargest for recommending top-rated movies

# Class representing a Movie entity
class Movie:
    id_counter = 1  # Class-level counter for movie IDs

    def __init__(self, title, genre, rating, language):
        self.id = Movie.id_counter  # Assign a unique ID to each movie
        self.title = title  # Title of the movie
        self.genre = genre  # Genre of the movie
        self.rating = rating  # Rating of the movie
        self.language = language  # Language of the movie
        Movie.id_counter += 1  # Increment the counter for the next movie

    def __repr__(self):
        return f"ID: {self.id} - {self.title} ({self.genre}) - Rating: {self.rating} - Language: {self.language}"

# Class implementing the Movie Recommendation System
class MovieRecommendationSystem:
    def __init__(self):
        # Pre-defined list of movies
        self.movies = [
            Movie("Rangasthalam", "Action", 8.5, "Telugu"),
            Movie("Bahubali: The Beginning", "Fantasy", 8.0, "Telugu"),
            Movie("Arjun Reddy", "Romance", 7.5, "Telugu"),
            Movie("Bheeshma", "Romantic Comedy", 7.2, "Telugu"),
            Movie("Inception", "Sci-Fi", 8.8, "English"),
            Movie("Interstellar", "Sci-Fi", 8.6, "English"),
            Movie("DDLJ", "Romance", 8.2, "Hindi"),
            Movie("Sholay", "Action", 8.7, "Hindi"),
            Movie("3 Idiots", "Comedy", 8.4, "Hindi")
        ]
        # Indexes for efficient querying
        self.genre_index = defaultdict(list)  # Dictionary to store movies indexed by genre
        self.title_index = {}  # Dictionary to store movies indexed by title
        self.id_index = {}  # Dictionary to store movies indexed by ID
        self.max_movies = 10  # Maximum number of movies allowed

        # Populate indexes and genre_index during initialization
        for movie in self.movies:
            self.title_index[movie.title.lower()] = movie
            self.genre_index[movie.genre.lower()].append(movie)
            self.id_index[movie.id] = movie

    # Method to add a new movie to the system
    def add_movie(self, title, genre, rating, language):
        if len(self.movies) >= self.max_movies:
            wx.MessageBox(f"Maximum limit of {self.max_movies} movies reached. Cannot add more movies.", "Error", wx.OK | wx.ICON_ERROR)
            return False  # Return False if max limit is reached

        # Create a new Movie object and add it to the respective indexes
        movie = Movie(title, genre, rating, language)
        self.movies.append(movie)
        self.title_index[title.lower()] = movie
        self.genre_index[genre.lower()].append(movie)
        self.id_index[movie.id] = movie  # Store movie in id_index with its ID as key

        wx.MessageBox(f"Movie '{title}' added successfully with ID: {movie.id}.", "Success", wx.OK | wx.ICON_INFORMATION)
        return True  # Return True on successful addition

    # Method to search for movies based on various criteria
    def search_movies(self, query):
        results = []
        
        # Search by title
        if query.lower() in self.title_index:
            results.append(self.title_index[query.lower()])
        
        # Search by genre
        results.extend(self.genre_index[query.lower()])

        # Search by language
        query_language = query.lower()
        language_results = [movie for movie in self.movies if query_language in movie.language.lower()]
        results.extend(language_results)

        # Search by movie ID
        try:
            movie_id = int(query)
            results.append(self.id_index.get(movie_id))
        except ValueError:
            pass
        
        # Search by highest ratings
        if query.lower() == "highest rating":
            results = nlargest(5, self.movies, key=lambda x: x.rating)
        
        # Remove None entries and duplicates
        results = [movie for movie in results if movie is not None]
        results = list(set(results))  # Remove duplicates

        return results  # Return search results

    # Method to recommend top-rated movies
    def recommend_movies(self, num_recommendations):
        return nlargest(num_recommendations, self.movies, key=lambda x: x.rating)

    # Method to delete a movie from the system
    def delete_movie(self, query):
        # Check if query is a movie title or movie object or movie ID
        if isinstance(query, str):
            movie = self.title_index.pop(query.lower(), None)
        elif isinstance(query, Movie):
            movie = query
        else:
            movie = self.id_index.get(query)

        if movie:
            self.movies.remove(movie)  # Remove movie from movies list
            self.genre_index[movie.genre.lower()].remove(movie)  # Remove from genre index
            self.id_index.pop(movie.id, None)  # Remove from ID index
            wx.MessageBox(f"Movie '{movie.title}' (ID: {movie.id}) deleted successfully.", "Success", wx.OK | wx.ICON_INFORMATION)

    # Method to display all movies in a message box
    def display_all_movies(self):
        all_movies_str = "\n".join([str(movie) for movie in self.movies])
        wx.MessageBox(f"All Movies:\n\n{all_movies_str}", "All Movies", wx.OK | wx.ICON_INFORMATION)

    # Method to add a new movie via a dialog window
    def add_new_movie(self):
        dialog = wx.Dialog(None, title="Add New Movie")

        panel = wx.Panel(dialog)
        vbox = wx.BoxSizer(wx.VERTICAL)

        wx.StaticText(panel, label="Enter Movie Details:")
        grid = wx.FlexGridSizer(4, 2, 5, 5)

        title_text = wx.TextCtrl(panel)
        grid.Add(wx.StaticText(panel, label="Title:"), 0, wx.EXPAND)
        grid.Add(title_text, 0, wx.EXPAND)

        genre_text = wx.TextCtrl(panel)
        grid.Add(wx.StaticText(panel, label="Genre:"), 0, wx.EXPAND)
        grid.Add(genre_text, 0, wx.EXPAND)

        rating_text = wx.TextCtrl(panel)
        grid.Add(wx.StaticText(panel, label="Rating:"), 0, wx.EXPAND)
        grid.Add(rating_text, 0, wx.EXPAND)

        language_text = wx.TextCtrl(panel)
        grid.Add(wx.StaticText(panel, label="Language:"), 0, wx.EXPAND)
        grid.Add(language_text, 0, wx.EXPAND)

        vbox.Add(grid, 0, wx.ALL | wx.EXPAND, 10)

        btn_ok = wx.Button(panel, label="Add Movie")
        btn_ok.Bind(wx.EVT_BUTTON, lambda event: self._on_add_movie(dialog, title_text.GetValue(), genre_text.GetValue(), rating_text.GetValue(), language_text.GetValue()))
        vbox.Add(btn_ok, 0, wx.ALL | wx.CENTER, 10)

        panel.SetSizer(vbox)

        dialog.ShowModal()  # Show the dialog window

    # Handler for adding a movie from the dialog window
    def _on_add_movie(self, dialog, title, genre, rating, language):
        try:
            rating = float(rating)
            added = self.add_movie(title, genre, rating, language)
            if added:
                dialog.Close()  # Close the dialog on successful addition
        except ValueError:
            wx.MessageBox("Rating should be a number (e.g., 8.5). Please enter a valid rating.", "Invalid Rating", wx.OK | wx.ICON_ERROR)

# Main application frame
class MovieRecommendationApp(wx.Frame):
    def __init__(self, parent, title):
        super(MovieRecommendationApp, self).__init__(parent, title=title, size=(800, 600))

        self.system = MovieRecommendationSystem()  # Initialize the movie recommendation system

        self.InitUI()  # Set up the user interface
        self.Centre()  # Center the frame on the screen
        self.Show()  # Display the frame

    # Method to initialize the user interface
    def InitUI(self):
        menubar = wx.MenuBar()  # Create a menu bar
        fileMenu = wx.Menu()  # Create a menu
        fileMenu.Append(wx.ID_EXIT, "Exit", "Exit application")  # Add an exit option to the menu
        menubar.Append(fileMenu, "&File")  # Append the menu to the menu bar
        self.SetMenuBar(menubar)  # Set the menu bar for the frame

        self.Bind(wx.EVT_MENU, self.OnExit, id=wx.ID_EXIT)  # Bind the exit option to OnExit method

        panel = wx.Panel(self)  # Create a panel for placing widgets
        vbox = wx.BoxSizer(wx.VERTICAL)  # Vertical box sizer for organizing widgets

        # Buttons for different functionalities
        btn_add_movie = wx.Button(panel, label="Add Movie")
        btn_add_movie.Bind(wx.EVT_BUTTON, lambda event: self._on_add_new_movie())
        vbox.Add(btn_add_movie, 0, wx.ALL | wx.EXPAND, 10)
                 
        # Add button for searching movies
        btn_search_movie = wx.Button(panel, label="Search Movies")
        btn_search_movie.Bind(wx.EVT_BUTTON, lambda event: self._on_search_movie())
        vbox.Add(btn_search_movie, 0, wx.ALL | wx.EXPAND, 10)

        # Add button for recommending movies
        btn_recommend_movies = wx.Button(panel, label="Recommend Movies")
        btn_recommend_movies.Bind(wx.EVT_BUTTON, lambda event: self._on_recommend_movies())
        vbox.Add(btn_recommend_movies, 0, wx.ALL | wx.EXPAND, 10)

        # Add button for deleting a movie
        btn_delete_movie = wx.Button(panel, label="Delete Movie")
        btn_delete_movie.Bind(wx.EVT_BUTTON, lambda event: self._on_delete_movie())
        vbox.Add(btn_delete_movie, 0, wx.ALL | wx.EXPAND, 10)

        # Add button for displaying all movies
        btn_display_all_movies = wx.Button(panel, label="Display All Movies")
        btn_display_all_movies.Bind(wx.EVT_BUTTON, lambda event: self._on_display_all_movies())
        vbox.Add(btn_display_all_movies, 0, wx.ALL | wx.EXPAND, 10)

        # Add button for exiting the application
        btn_exit_app = wx.Button(panel, label="Exit Application")
        btn_exit_app.Bind(wx.EVT_BUTTON, self.confirm_exit)  # Bind to confirm_exit method
        vbox.Add(btn_exit_app, 0, wx.ALL | wx.EXPAND, 10)

        panel.SetSizer(vbox)  # Set the vertical box sizer for the panel

    # Method to handle adding a new movie
    def _on_add_new_movie(self):
        self.system.add_new_movie()  # Call add_new_movie method from MovieRecommendationSystem

    # Method to handle searching for movies
    def _on_search_movie(self):
        dialog = wx.TextEntryDialog(self, "Enter movie title, genre, language or ID to search:", "Search Movies")
        if dialog.ShowModal() == wx.ID_OK:
            query = dialog.GetValue()
            results = self.system.search_movies(query)
            if results:
                result_str = "\n\n".join([str(movie) for movie in results])
                wx.MessageBox(f"Search Results:\n\n{result_str}", "Search Results", wx.OK | wx.ICON_INFORMATION)
        dialog.Destroy()  # Destroy the dialog after use

    # Method to handle recommending movies
    def _on_recommend_movies(self):
        dialog = wx.TextEntryDialog(self, "Enter number of movies to recommend:", "Recommend Movies")
        if dialog.ShowModal() == wx.ID_OK:
            try:
                num_recommendations = int(dialog.GetValue())
                recommendations = self.system.recommend_movies(num_recommendations)
                if recommendations:
                    result_str = "\n\n".join([str(movie) for movie in recommendations])
                    wx.MessageBox(f"Recommended Movies:\n\n{result_str}", "Recommendations", wx.OK | wx.ICON_INFORMATION)
            except ValueError:
                wx.MessageBox("Please enter a valid number.", "Invalid Input", wx.OK | wx.ICON_ERROR)
        dialog.Destroy()  # Destroy the dialog after use

    # Method to handle deleting a movie
    def _on_delete_movie(self):
        dialog = wx.TextEntryDialog(self, "Enter movie title, genre, language or ID to delete:", "Delete Movie")
        if dialog.ShowModal() == wx.ID_OK:
            query = dialog.GetValue()
            self.system.delete_movie(query)
        dialog.Destroy()  # Destroy the dialog after use

    # Method to handle displaying all movies
    def _on_display_all_movies(self):
        self.system.display_all_movies()  # Call display_all_movies method from MovieRecommendationSystem

    # Method to confirm application exit
    def confirm_exit(self, event):
        dialog = wx.MessageDialog(self, "Are you sure you want to exit?", "Confirm Exit", wx.YES_NO | wx.ICON_QUESTION)
        response = dialog.ShowModal()
        dialog.Destroy()

        if response == wx.ID_YES:
            self.Close()  # Close the application

    # Method to handle application exit
    def OnExit(self, event):
        self.Close()  # Close the application

def main():
    app = wx.App()  # Create an application instance
    MovieRecommendationApp(None, title="Welcome To SP's Movie Recommendation System")  # Create an instance of MovieRecommendationApp
    app.MainLoop()  # Start the application main loop

if __name__ == "__main__":
    main()  # Entry point of the application
         
